In [ ]:
import pandas as pd 
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
import os 
import math
import json

In [ ]:
DATASET_PATH ="./content/drive/MyDrive/clinical/segmented"
JSON_PATH = "data.json"
SAMPLE_RATE =22050
DURATION = 1 
SAMPLE_PER_TRACK = SAMPLE_RATE*DURATION

In [ ]:
len(DATASET_PATH)

43

In [ ]:
def save_mffc (dataset_path ,json_path , n_mfcc=13 , n_fft=2048, hop_length=512 , num_segment=5):

  data = {
      "mapping":[],
      "mfcc":[],
      "labels":[]
  }


  num_sample_per_segment = int(SAMPLE_PER_TRACK/num_segment)
  expected_num_mfcc_per_segment = math.ceil(num_sample_per_segment / hop_length)


  for i , (dirpath,dirnames,filenames) in enumerate(os.walk(dataset_path)):

    if dirpath is not dataset_path:

      dirpath_components = dirpath.split("/")
      semantic_labels = dirpath_components[:-1]
      data["mapping"].append(semantic_labels)
      print("\nProcessing {}".format(semantic_labels))


      for f in filenames:

        file_path = os.path.join(dirpath , f)

        signal ,sr  = librosa.load(file_path , sr=SAMPLE_RATE)


        for s in range(num_segment):
          start_sample  = num_sample_per_segment*s
          finish_sample = start_sample + num_sample_per_segment

          mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample] , 
                                      sr=SAMPLE_RATE  ,
                                      n_fft =  n_fft ,
                                      n_mfcc=n_mfcc ,
                                      hop_length = hop_length) 
          mfcc = mfcc.T 

          if len(mfcc) == expected_num_mfcc_per_segment:

            data["mfcc"].append(mfcc.tolist())
            data["labels"].append(i-1)
            print("{} , segment : {}".format(file_path ,s))



  with open(json_path , "w") as fp:
    json.dump(data ,fp , indent=4)


In [ ]:
save_mffc(DATASET_PATH ,JSON_PATH , num_segment=5)